## signate 画像分類コンペ(2クラス)

In [89]:
# インポート
import glob
import random
import pickle

import tqdm
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import get_cosine_schedule_with_warmup
from IPython.display import clear_output
# import torchvision.models as models
# from torchvision.models import EfficientNet_V2_L_Weights

### 初期処理

In [90]:
BASE_PATH = 'C:\\Users\\zigza\\GitFile\\signate\\package_analysis\\'
DATA_PATH = BASE_PATH + 'datasets\\'
TRAIN_PATH = DATA_PATH + 'train\\'
TEST_PATH = DATA_PATH + 'test\\'
OUT_PATH = BASE_PATH + 'out\\'
# C:\\Users\\zigza\\GitFile\\signate\\package_analysis\\train.csv
# C:\Users\zigza\GitFile\signate\package_analysis\datasets\train.csv

In [91]:
BASE_PATH = '/home/masa1357/Dockerdata/gitfile/package_analysis/'
DATA_PATH = BASE_PATH + 'datasets/'
TRAIN_PATH = DATA_PATH + 'train/'
TEST_PATH = DATA_PATH + 'test/'
OUT_PATH = BASE_PATH + 'out/'

In [92]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')
train_df.head()


,image_name,label
0,0000.png,0
1,0001.png,1
2,0002.png,1
3,0003.png,1
4,0004.png,0


In [93]:
# seedの固定
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 0
fix_seed(SEED)

### Datamodule

In [94]:
class MyDataset(Dataset):

    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        self.img_path = TRAIN_PATH

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        # 画像を読みこんで、指定の方法でtransform
        img_name = os.path.join(self.img_path, self.file_list.iloc[index,0])
        img = Image.open(img_name)
        img_transformed = self.transform(img)
        label = int(self.file_list.iloc[index,1])

        return img_transformed, label

In [95]:
class CreateDataModule(pl.LightningDataModule):

    def __init__(self, train_df, val_df, test_df, img_size=224,
                 mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),
                 batch_size=16):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.batch_size = batch_size

        # train時、val/test時の前処理をそれぞれ定義
        self.train_transforms = transforms.Compose([
            transforms.RandomResizedCrop(img_size, scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

        self.val_test_transforms = transforms.Compose([
            transforms.Resize(img_size),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

    # データのダウンロードなどを行う場合は定義、今回は不要
    def prepare_data(self):
        pass

    # Trainer.fit()ではtrain/valのDatasetを、Trainer.test()ではtestのDatasetを生成
    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = MyDataset(self.train_df, self.train_transforms)
            self.val_dataset = MyDataset(self.val_df, self.val_test_transforms)

        if stage == 'test' or stage is None:
            self.test_dataset = MyDataset(self.test_df, self.val_test_transforms)

    # こちらもTrainer.fit()ではtrain/valのDataLoaderを、Trainer.test()ではtestのDataLoaderを生成
    # trainはshuffleあり、val/testはshuffleなし
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


In [96]:
# seedを固定
fix_seed(SEED)

#train_df内のデータを7:2:1の割合でval_df,test_dfに分割
train_df, val_df = train_test_split(train_df, test_size=0.3, shuffle=True, random_state=SEED)
val_df, test_df = train_test_split(val_df, test_size=0.33, shuffle=True, random_state=SEED)

# インスタンスを作成
data_module = CreateDataModule(train_df,val_df,test_df)

### Classifier

In [102]:
class ImageClassifier(pl.LightningModule):
    def __init__(self, model_name, n_classes, lr=0.0001, criterion=torch.nn.CrossEntropyLoss()):
        super().__init__()
        self.save_hyperparameters()

        # timmで学習済みモデルをダウンロードし、classifier部分を付替え
        # n_classesにはラベルの件数を渡す（今回はアリとハチの2つなので2）
        self.model = timm.create_model(model_name, pretrained=False)
        #self.model = models.efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.ImageNet1K_V1, progress=True)
        #self.model = models.efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.IMAGENET1K_V1)
        self.model.classifier = nn.Linear(self.model.conv_head.out_channels, n_classes)

        self.lr = lr
        self.criterion = criterion
        self.outputs = []

    # 順伝搬
    def forward(self, imgs, labels=None):
        preds = self.model(imgs)
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)
        return loss, preds

    # trainのミニバッチに対して行う処理
    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        loss, preds = self.forward(imgs=imgs, labels=labels)
        return {'loss': loss, 'batch_preds': preds.detach(), 'batch_labels': labels.detach()}

    # validation、testでもtrain_stepと同じ処理を行う
    def validation_step(self, batch, batch_idx):
        result = self.training_step(batch, batch_idx)
        self.outputs.append(result)
        return result

    def test_step(self, batch, batch_idx):
        result = self.training_step(batch, batch_idx)
        self.outputs.append(result)
        return result

    # epoch終了時にvalidationのlossとaccuracyを記録
    def on_validation_epoch_end(self):

        # loss計算
        epoch_preds = torch.cat([x['batch_preds'] for x in self.outputs])
        epoch_labels = torch.cat([x['batch_labels'] for x in self.outputs])
        epoch_loss = self.criterion(epoch_preds, epoch_labels)
        self.log(f"val_loss", epoch_loss, logger=True)

        # accuracy計算
        num_correct = (epoch_preds.argmax(dim=1) == epoch_labels).sum().item()
        epoch_accuracy = num_correct / len(epoch_labels)
        self.log(f"val_accuracy", epoch_accuracy, logger=True)
        self.outputs = []


    def on_test_epoch_end(self):
        ret = self.on_validation_epoch_end()
        return ret

    def configure_optimizers(self):
        optimizer = optim.AdamW(lr=self.lr, params=self.model.parameters())
        
        # schedulerの設定（どっちか）
        #scheduler = {'scheduler': optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)}
        warmup_steps = int(0.1 * self.trainer.max_steps)  # warmupのステップ数を全ステップの10%に設定
        scheduler = {
            'scheduler': get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=self.trainer.max_steps),
            'interval': 'step',
            'frequency': 1
        }



### Train/Validation

In [103]:
# モデルインスタンスの作成
model = ImageClassifier(model_name="efficientnetv2_s", n_classes=2)

In [104]:
# EarlyStoppingの設定
# 3epochで'val_loss'が0.05以上減少しなければ学習をストップ
early_stop_callback = EarlyStopping(
    monitor='val_loss', min_delta=0.05, patience=3, mode='min')

# モデルの保存先
# epoch数に応じて、「epoch=0.ckpt」のような形で保存
checkpoint_callback = ModelCheckpoint(
    dirpath=OUT_PATH + 'model_checkpoints/',
    filename='{epoch}', monitor='val_loss', mode='min', verbose=True)

# trainerの設定
trainer = pl.Trainer(max_epochs=10,
                     devices=1,
                     callbacks=[checkpoint_callback, early_stop_callback],
                     log_every_n_steps=10)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [105]:
# gpuを設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 訓練開始
data_module.setup('fit')
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | EfficientNet     | 20.2 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.720    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 96: 'val_loss' reached 0.72588 (best 0.72588), saving model to '/home/masa1357/Dockerdata/gitfile/package_analysis/out/model_checkpoints/epoch=0-v1.ckpt' as top 1


...

Epoch 1, global step 192: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 2, global step 288: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 3, global step 384: 'val_loss' was not in top 1


In [106]:
# 精度検証
data_module = CreateDataModule(train_df,val_df,test_df)
data_module.setup('test')
result = trainer.test(datamodule=data_module,ckpt_path=checkpoint_callback.best_model_path)
result

Restoring states from the checkpoint path at /home/masa1357/Dockerdata/gitfile/package_analysis/out/model_checkpoints/epoch=0-v1.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /home/masa1357/Dockerdata/gitfile/package_analysis/out/model_checkpoints/epoch=0-v1.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.49074074625968933
        val_loss            0.7408518195152283
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.7408518195152283, 'val_accuracy': 0.49074074625968933}]

In [107]:
# tensorboardでの確認
%load_ext tensorboard
#%tensorboard --logdir C:/Users/zigza/GitFile/signate/package_analysis/code/lightning_logs
%tensorboard --logdir /home/masa1357/Dockerdata/gitfile/package_analysis/code/lightning_logs

In [ ]:
# 最良モデルの保存
best_model = ImageClassifier.load_from_checkpoint(checkpoint_callback.best_model_path)

with open('./best_model.pkl', mode='wb') as fp:
    pickle.dump(best_model, fp)